### Imports

In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
import re
import datetime
import requests
from bs4 import BeautifulSoup

# Caminho da pasta "Twitch"
base_dir = Path(os.getcwd()).resolve().parent  # Ajuste se necessário

# Adicionar as pasta ao sys.path
data_dir = base_dir / 'data'
src_data_dir = base_dir / 'src' / 'data'
sys.path.append(str(data_dir))
sys.path.append(str(src_data_dir))

try:
    import API_Spotify as api
    import get_data
except ImportError:
    print("Erro: Não foi possível importar algum módulo. Verifique se o caminho está correto.")

C:\Users\diogo\Desktop\BDDA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get Data

In [2]:
# Diretório onde vão ficar os datasets
data_directory = data_dir / "raw"

def check_directory(data_directory):
    if os.path.exists(data_directory) and os.path.isdir(data_directory):
        if not os.listdir(data_directory):  # Verifica se a lista de ficheiros/diretórios está vazia
            print("A pasta está vazia.")
            return True
        else:
            print("A pasta não está vazia.")
            return False
    else:
        print("A pasta não existe ou não é um diretório.")
        return True

In [3]:
if check_directory(data_directory):
    # Lista de links dos datasets no Kaggle
    links = [
        "hedizekri/top-charts-artists-country",  # Países dos artistas
        "jackharding/spotify-artist-metadata-top-10k",  # Idade, género dos artistas
        "rodolfofigueroa/spotify-12m-songs",  # Spotify 12M Songs
        "estienneggx/spotify-unpopular-songs",  # Unpopular Songs
    ]

    # Executa o processo de download e conversão
    get_data.kaggleDownload(data_directory, links)

A pasta não está vazia.


## Artists

In [4]:
Artist_country =  pd.read_csv(data_directory / "Top_Charts_Artists_Country.csv")
Artist_metaData = pd.read_csv(data_directory / "top10k-spotify-artist-metadata.csv")
# Artist_tracks = pd.read_csv(data_directory /"tracks_features.csv")
Artist_country

,Artist,Country
0,Miley Cyrus,United States
1,FIFTY FIFTY,United Kingdom
2,ROSALÍA,Spain
3,PinkPantheress,United Kingdom
4,Metro Boomin,United States
...,...,...
960,Meck,United Kingdom
961,Matt Simons,United States
962,Hoodie Forester,Netherlands
963,NSB,United States


In [5]:
Artist_metaData

,Unnamed: 0,index,artist,gender,age,type,country,city_1,district_1,city_2,district_2,city_3,district_3
0,0,0,Drake,male,33,person,CA,NaN,NaN,Toronto,NaN,NaN,NaN
1,1,1,Post Malone,male,25,person,US,NaN,NaN,Syracuse,NaN,NaN,NaN
2,2,2,Ed Sheeran,male,29,person,GB,NaN,NaN,Halifax,NaN,NaN,NaN
3,3,3,J Balvin,male,35,person,CO,NaN,NaN,Medellín,NaN,NaN,NaN
4,4,4,Bad Bunny,male,26,person,PR,NaN,NaN,San Juan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,4991,si_el_bien,NaN,0,NaN,NaN,Chicago,NaN,NaN,NaN,NaN,NaN
9996,9996,4992,Antonello Venditti,male,71,person,IT,NaN,NaN,Rome,NaN,NaN,NaN
9997,9997,4993,Lea Salonga,female,49,person,PH,NaN,NaN,NaN,NaN,NaN,NaN
9998,9998,4994,Vertical Horizon,mixed,29,group,NaN,Boston,NaN,NaN,NaN,NaN,NaN


### Limpeza

In [6]:
def clean_text(text):
    if pd.isnull(text):  # Verifica se é nulo
        return text
    # Remove pontuação usando regex
    text = re.sub(r'[^\w\s]', '', text)
    # Converte para minúsculas
    text = text.title()
    # Remove espaços extras
    text = ' '.join(text.split())
    return text

# Colocar nomes uniformizados
Artist_metaData['artist_cleaned'] = Artist_metaData['artist'].apply(clean_text)
Artist_country['artist_cleaned'] = Artist_country['Artist'].apply(clean_text)

In [7]:
# Artistas que não estão presentes na DB secundária (vão ser apagados)
Artist_country[~Artist_country["artist_cleaned"].isin(Artist_metaData["artist_cleaned"])]

,Artist,Country,artist_cleaned
1,FIFTY FIFTY,United Kingdom,Fifty Fifty
3,PinkPantheress,United Kingdom,Pinkpantheress
5,David Kushner,United States,David Kushner
7,Rema,Nigeria,Rema
9,d4vd,United States,D4Vd
...,...,...,...
958,Big Brovaz,United Kingdom,Big Brovaz
960,Meck,United Kingdom,Meck
962,Hoodie Forester,Netherlands,Hoodie Forester
963,NSB,United States,Nsb


In [8]:
# Juntar as tabelas
if 'Artist_metaData' in locals() and 'Artist_country' in locals():
    # Faz o merge entre as duas tabelas com base na coluna 'artist_cleaned'
    merged_table = pd.merge(
        Artist_metaData,
        Artist_country,
        on='artist_cleaned',  # Coluna de ligação
        how='inner'           # Tipo de junção ('inner', 'left', 'right', 'outer')
    )
    del Artist_metaData, Artist_country

# Seleciona apenas as colunas que nos interessam
Artist_details = merged_table[['artist_cleaned', 'gender', 'age', 'Country']]
Artist_details = Artist_details.rename(columns={'artist_cleaned': 'artist_name', 'Country': 'country_born'})

# Queremos artistas singulares, não grupos
Artist_details = Artist_details[Artist_details['gender'] != 'mixed']

# A base de dados com a idade dos artistas foi publicada em 2021
## https://www.kaggle.com/datasets/jackharding/spotify-artist-metadata-top-10k/data
if 0 in Artist_details["age"].unique():
    Artist_details["age"] = Artist_details["age"] + datetime.datetime.now().year - 2021
Artist_details

,artist_name,gender,age,country_born
0,Drake,male,37,Canada
1,Post Malone,male,29,United States
2,Ed Sheeran,male,33,United Kingdom
3,J Balvin,male,39,Colombia
4,Bad Bunny,male,30,Puerto Rico
...,...,...,...,...
651,Mo,NaN,4,United Kingdom
660,Sisqo,male,45,United States
664,Waka Flocka Flame,male,38,United States
665,Lou Bega,male,49,Germany


In [9]:
Artist_details[Artist_details.isnull().any(axis=1)]

,artist_name,gender,age,country_born
53,Annemarie,NaN,4,France
314,Bizarrap,NaN,26,Argentina
357,Big Shaq,NaN,32,United Kingdom
388,Giveon,NaN,4,United States
399,Nsg,NaN,4,Indonesia
424,Mo,NaN,4,United Kingdom
508,Chip,NaN,4,United Kingdom
587,Lyodra,NaN,4,Indonesia
602,Diskoria,NaN,4,Indonesia
651,Mo,NaN,4,United Kingdom


In [10]:
# Definir manualmente os valores da coluna 'gender' para alguns artistas
Artist_details.loc[Artist_details['artist_name'] == 'Annemarie', 'gender'] = 'female'
Artist_details.loc[Artist_details['artist_name'] == 'Bizarrap', 'gender'] = 'Male'
Artist_details.loc[Artist_details['artist_name'] == 'Big Shaq', 'gender'] = 'Male'
Artist_details.loc[Artist_details['artist_name'] == 'Giveon', 'gender'] = 'Male'
Artist_details.loc[Artist_details['artist_name'] == 'Chip', 'gender'] = 'Male'
Artist_details.loc[Artist_details['artist_name'] == 'Lyodra', 'gender'] = 'Female'

# Apagar colunas com valores nulos
Artist_details = Artist_details.dropna(axis=0)

In [11]:
# Definir manualmente os valores da coluna 'age' para alguns artistas
Artist_details.loc[Artist_details['artist_name'] == 'Annemarie', 'age'] = 33
Artist_details.loc[Artist_details['artist_name'] == 'Bizarrap', 'age'] = 26
Artist_details.loc[Artist_details['artist_name'] == 'Big Shaq', 'age'] = 33
Artist_details.loc[Artist_details['artist_name'] == 'Giveon', 'age'] = 29
Artist_details.loc[Artist_details['artist_name'] == 'Chip', 'age'] = 34
Artist_details.loc[Artist_details['artist_name'] == 'Lyodra', 'age'] = 21
Artist_details.loc[Artist_details['artist_name'] == 'Joel Adams', 'age'] = 28

In [12]:
incorretAge = Artist_details[Artist_details["age"] == datetime.datetime.now().year - 2021][["artist_name", "age"]]
incorretAge

,artist_name,age
50,Manuel Turizo,4
66,Offset,4
85,Paulo Londra,4
93,Tones And I,4
109,Rauw Alejandro,4
124,Lil Pump,4
157,Meduza,4
174,Nio Garcia,4
183,Dynoro,4
188,Saint Jhn,4


In [13]:
def is_convertible_to_int(s):
    try:
        int(s)  # Tenta converter para int
        return True  # Se não der erro, é conversível
    except ValueError:
        return False  # Caso contrário, não é


In [14]:
# Função para obter a data de nascimento de um artista na Wikipedia
def get_artist_birth_date(artist_name):
    # URL da Wikipedia do artista
    url = f"https://en.wikipedia.org/wiki/{artist_name.replace(' ', '_')}"
    response = requests.get(url)

    if response.status_code == 200:
        # Parseando o conteúdo HTML da página
        soup = BeautifulSoup(response.content, "html.parser")

        # Procurando a infobox com as informações biográficas
        infobox = soup.find("table", {"class": "infobox"})

        if infobox:
            # Procurando pela data de nascimento
            for row in infobox.find_all("tr"):
                header = row.find("th")
                if header and "Born" in header.text:
                    # Encontrar a data de nascimento na célula da tabela
                    birth_date_cell = row.find("td")
                    # print(birth_date_cell)
                    if birth_date_cell:
                        birth_date = birth_date_cell.text.strip()
                        birth_date_split = birth_date.split(' ')
                        for i in birth_date_split:
                            if "(" in i and ")" in i and "-" in i:
                                age = int(i[i.find("(") + 1:i.find("-")])
                                return age
                        if is_convertible_to_int(birth_date_split[0]):
                            return int(datetime.datetime.now().year) - int(birth_date_split[0])
                        return None
    return None

# Função para calcular a idade com base na data de nascimento
def calculate_age(birth_date):
    try:
        # Tentando extrair o ano da data de nascimento (formato geralmente: 12 January 1990)
        birth_year = int(birth_date)  # Extrair o ano
        current_year = datetime.datetime.now().year
        return int(current_year - birth_year)
    except Exception as e:
        print(f"Erro ao calcular a idade: {e}")
        return None

# Função para atualizar as idades na tabela Artist_details
def update_artist_ages(df):
    ages = []  # Lista para armazenar as idades

    for artist_name in df['artist_name']:
        birth_date = get_artist_birth_date(artist_name)
        if birth_date:
            age = calculate_age(birth_date)
            ages.append(int(age))
        else:
            ages.append(None)  # Caso não consiga encontrar a idade

    # Atualizar a coluna 'age' com as idades obtidas
    df['age'] = ages
    return df

# Atualizar as idades na tabela
incorretAge = update_artist_ages(incorretAge)

# Atualizar a coluna 'age' na tabela principal
Artist_details = Artist_details.merge(
    incorretAge,  # Tabela secundária
    on='artist_name',  # Coluna de ligação
    how='left',  # Faz uma junção à esquerda
    suffixes=('', '_secondary')  # Adiciona sufixos para diferenciar colunas
)

# Preencher os valores de 'age' com os dados da tabela secundária, se disponíveis
Artist_details['age'] = Artist_details['age_secondary'].combine_first(Artist_details['age'])

# Remover a coluna auxiliar
Artist_details = Artist_details.drop(columns=['age_secondary'])

Artist_details.dropna(inplace=True)

Artist_details

,artist_name,gender,age,country_born
0,Drake,male,37.0,Canada
1,Post Malone,male,29.0,United States
2,Ed Sheeran,male,33.0,United Kingdom
3,J Balvin,male,39.0,Colombia
4,Bad Bunny,male,30.0,Puerto Rico
...,...,...,...,...
487,Miguel,male,54.0,United States
488,Sisqo,male,45.0,United States
489,Waka Flocka Flame,male,38.0,United States
490,Lou Bega,male,49.0,Germany


In [15]:
Artist_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   artist_name   492 non-null    object 
 1   gender        492 non-null    object 
 2   age           492 non-null    float64
 3   country_born  492 non-null    object 
dtypes: float64(1), object(3)
memory usage: 15.5+ KB


In [16]:
Artist_details.describe()

,age
count,492.000000
mean,36.215447
std,11.861142
min,4.000000
25%,30.000000
50%,36.000000
75%,43.000000
max,96.000000


In [ ]:
# TODO Corrigir estes nomes

# Nome fornecido: J Cole, Nome retornado pela API: J. Cole
# Nome fornecido: Zayn, Nome retornado pela API: Niall Horan
# Nome fornecido: Mø, Nome retornado pela API: Morad
# Nome fornecido: Geazy, Nome retornado pela API: Geany Morandi
# Nome fornecido: Aap Rocky, Nome retornado pela API: A$AP Rocky
# Nome fornecido: Gigi Dagostino, Nome retornado pela API: Gigi D'Agostino
# Nome fornecido: Playboi Carti, Nome retornado pela API: Plaboi Cari
# Nome fornecido: Pnk, Nome retornado pela API: pnkcité
# Nome fornecido: Machine Gun Kelly, Nome retornado pela API: mgk
# Nome fornecido: Tyler The Creator, Nome retornado pela API: Tyler, The Creator
# Nome fornecido: Dram, Nome retornado pela API: DRAMA
# Nome fornecido: Mr Probz, Nome retornado pela API: Mr. Probz
# Nome fornecido: Russ, Nome retornado pela API: Russ Millions
# Nome fornecido: Aap Ferg, Nome retornado pela API: Jorge Ferreira & Pega
# Nome fornecido: Neyo, Nome retornado pela API: Ne-Yo
# Nome fornecido: Marc E Bassy, Nome retornado pela API: Marc E. Bassy
# Nome fornecido: Ti, Nome retornado pela API: TINI
# Nome fornecido: Rob Tone, Nome retornado pela API: Rob $tone
# Nome fornecido: Ruth B, Nome retornado pela API: Ruth B.
# Nome fornecido: Mk, Nome retornado pela API: Mk.gee
# Nome fornecido: Sebastian Ingrosso, Nome retornado pela API: Axwell /\ Ingrosso
# Nome fornecido: Nelly, Nome retornado pela API: Nelly Furtado
# Nome fornecido: Ot Genasis, Nome retornado pela API: O.T. Genasis
# Nome fornecido: The Notorious Big, Nome retornado pela API: The Notorious B.I.G.
# Nome fornecido: Eo, Nome retornado pela API: ÉoDan
# Nome fornecido: Philip George, Nome retornado pela API: Philippe Entremont
# Nome fornecido: Bob, Nome retornado pela API: Bob Sinclar
# Nome fornecido: Bob, Nome retornado pela API: Bob Sinclar
# Nome fornecido: Dr Dre, Nome retornado pela API: Dr. Dre
# Nome fornecido: Marina, Nome retornado pela API: Marina Sena
# Nome fornecido: Thedream, Nome retornado pela API: Ramones
# Nome fornecido: Marina, Nome retornado pela API: Marina Sena
# Nome fornecido: Mia, Nome retornado pela API: M.I.A.
# Nome fornecido: Mary J Blige, Nome retornado pela API: Mary J. Blige
# Nome fornecido: Jojo, Nome retornado pela API: Jojo's ASMR

### Api

In [17]:
# Demora, aproximadamente 0.26626 segundos por artista

# Problemas:
## PlayBoi Carti
api.get_artist_details(Artist_details, 'artist_name')
Artist_details[Artist_details.isnull().any(axis=1)]

Nome fornecido: J Cole, Nome retornado pela API: J. Cole
Nome fornecido: Zayn, Nome retornado pela API: Niall Horan
Nome fornecido: Mø, Nome retornado pela API: Morad
Nome fornecido: Geazy, Nome retornado pela API: Geany Morandi
Nome fornecido: Aap Rocky, Nome retornado pela API: A$AP Rocky
Nome fornecido: Gigi Dagostino, Nome retornado pela API: Gigi D'Agostino
Nome fornecido: Playboi Carti, Nome retornado pela API: Plaboi Cari
Nome fornecido: Pnk, Nome retornado pela API: pnkcité
Nome fornecido: Machine Gun Kelly, Nome retornado pela API: mgk
Nome fornecido: Tyler The Creator, Nome retornado pela API: Tyler, The Creator
Nome fornecido: Dram, Nome retornado pela API: DRAMA
Nome fornecido: Mr Probz, Nome retornado pela API: Mr. Probz
Nome fornecido: Russ, Nome retornado pela API: Russ Millions
Nome fornecido: Aap Ferg, Nome retornado pela API: Jorge Ferreira & Pega
Nome fornecido: Neyo, Nome retornado pela API: Ne-Yo
Nome fornecido: Marc E Bassy, Nome retornado pela API: Marc E. Bassy


,artist_name,gender,age,country_born,artist_id,followers,popularity,genres,image_url
54,J Cole,male,39.0,United States,J. Cole,NaN,NaN,None,None
63,Zayn,male,31.0,United Kingdom,Niall Horan,NaN,NaN,None,None
71,Mø,female,35.0,Denmark,Morad,NaN,NaN,None,None
79,Geazy,male,35.0,United States,Geany Morandi,NaN,NaN,None,None
93,Aap Rocky,male,35.0,United States,A$AP Rocky,NaN,NaN,None,None
162,Gigi Dagostino,male,56.0,Italy,Gigi D'Agostino,NaN,NaN,None,None
166,Yg,male,34.0,United States,7iqXu8imQ1rePIk5Nx1ctV,5.0,0.0,,None
172,Playboi Carti,male,27.0,United States,Plaboi Cari,NaN,NaN,None,None
175,Pnk,female,44.0,United States,pnkcité,NaN,NaN,None,None
193,Machine Gun Kelly,male,34.0,United States,mgk,NaN,NaN,None,None


In [18]:
Artist_details.dropna(inplace=True)
Artist_details

,artist_name,gender,age,country_born,artist_id,followers,popularity,genres,image_url
0,Drake,male,37.0,Canada,3TVXtAsR1Inumwj472S9r4,94870090.0,97.0,"canadian hip hop, canadian pop, hip hop, pop r...",https://i.scdn.co/image/ab6761610000e5eb429338...
1,Post Malone,male,29.0,United States,246dkjvS1zLTtiykXe5h60,46189374.0,90.0,"dfw rap, melodic rap, pop, rap",https://i.scdn.co/image/ab6761610000e5ebe17c0a...
2,Ed Sheeran,male,33.0,United Kingdom,6eUKZXaKkcviH0Ku9w2n3V,118264599.0,90.0,"pop, singer-songwriter pop, uk pop",https://i.scdn.co/image/ab6761610000e5eb784daf...
3,J Balvin,male,39.0,Colombia,1vyhD5VmyZ7KMfW5gqLgo5,37646369.0,88.0,"reggaeton, reggaeton colombiano, trap latino, ...",https://i.scdn.co/image/ab6761610000e5ebdea087...
4,Bad Bunny,male,30.0,Puerto Rico,4q3ewBCX7sLwd24euuV69X,88407015.0,96.0,"reggaeton, trap latino, urbano latino",https://i.scdn.co/image/ab6761610000e5eb744a42...
...,...,...,...,...,...,...,...,...,...
487,Miguel,male,54.0,United States,360IAlyVv4PCEVjgyMZrxK,5507641.0,81.0,"r&b, urban contemporary",https://i.scdn.co/image/ab6761610000e5eb35e36c...
488,Sisqo,male,45.0,United States,6x9QLdzo6eBZxJ1bHsDkjg,1099210.0,60.0,"contemporary r&b, hip pop, r&b, urban contempo...",https://i.scdn.co/image/65c4f2d897d65c2236ddc7...
489,Waka Flocka Flame,male,38.0,United States,6f4XkbvYlXMH0QgVRzW0sM,2743718.0,68.0,"atl hip hop, dirty south rap, pop rap, rap, so...",https://i.scdn.co/image/ab6761610000e5eb098d90...
490,Lou Bega,male,49.0,Germany,46lnlnlU0dXTDpoAUmH6Qx,142729.0,61.0,latin pop,https://i.scdn.co/image/ab6761610000e5ebcbe7dd...


## Popular Tracks

In [19]:
popular = pd.read_csv(data_directory / 'Spotify Most Streamed Songs.csv')
popular

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\diogo\\Desktop\\BDDA\\data\\raw\\Spotify Most Streamed Songs.csv'

In [6]:
popular.columns

Index(['track_name', 'artist(s)_name', 'artist_count', 'released_year',
       'released_month', 'released_day', 'in_spotify_playlists',
       'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts',
       'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm',
       'key', 'mode', 'danceability_%', 'valence_%', 'energy_%',
       'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%',
       'cover_url'],
      dtype='object')

In [ ]:
popular = popular.drop(columns=["artist_count", "released_year", 'released_month', 'released_day', 'in_spotify_playlists',
       'in_spotify_charts'])

In [13]:
popular.dtypes

track_name              object
artist(s)_name          object
artist_count             int64
released_year            int64
released_month           int64
released_day             int64
in_spotify_playlists     int64
in_spotify_charts        int64
streams                 object
in_apple_playlists       int64
in_apple_charts          int64
in_deezer_playlists     object
in_deezer_charts         int64
in_shazam_charts        object
bpm                      int64
key                     object
mode                    object
danceability_%           int64
valence_%                int64
energy_%                 int64
acousticness_%           int64
instrumentalness_%       int64
liveness_%               int64
speechiness_%            int64
cover_url               object
dtype: object

## Unpopular Tracks

In [7]:
unpopular = pd.read_csv(data_directory / 'unpopular_songs.csv')
unpopular

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,explicit,popularity,track_name,track_artist,track_id
0,0.722,0.476,5,-8.203,1,0.0896,0.67000,0.0000,0.0926,0.9270,151.968,220549,False,8,Este Corazón,El Javi,0Z4UqiPI5c4dMJ5iPyO61d
1,0.955,0.281,0,-11.392,0,0.4660,0.00480,0.0595,0.1220,0.0934,127.991,149543,True,0,thinking and doing,Br80,568FGhLEnSZhl82t0YmQ96
2,0.753,0.355,8,-11.799,1,0.5140,0.57300,0.0000,0.0735,0.6000,91.978,130019,True,0,Puzzlement,JpThaRuler,4Lex8QZVkptfw2PZtjoXlp
3,0.650,0.331,3,-10.818,0,0.3830,0.86500,0.0000,0.1050,0.5810,99.513,116193,True,0,Sensi,JpThaRuler,4pMqXl6EfmZrodPMYHUq9Z
4,0.690,0.360,4,-15.624,0,0.4550,0.76400,0.0000,0.1810,0.1050,89.893,111827,True,0,Rass,JpThaRuler,2pvQEKHtlkHb6ijMwcMuKV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4068,0.862,0.578,10,-9.364,0,0.4430,0.01460,0.0000,0.1190,0.3850,81.059,139442,True,0,No Competition,T-Tomp,35ebaAwvPLjeTVUMQhyyqW
4069,0.686,0.511,8,-10.253,0,0.4350,0.01010,0.0000,0.1100,0.4020,137.768,97440,True,0,Test Me,T-Tomp,5WYoA9mhHWSyHkaGI3qKsO
4070,0.725,0.555,8,-13.018,1,0.3910,0.00189,0.0000,0.1100,0.1390,139.806,91742,True,0,Poof,T-Tomp,7aAsvABCFbZa7FsolZTASW
4071,0.763,0.534,1,-8.889,1,0.3170,0.00531,0.0000,0.1110,0.3020,82.507,113868,True,0,Right Now,T-Tomp,4c49ADGqWfJkA1vB45oXVG


In [8]:
unpopular.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'explicit', 'popularity', 'track_name', 'track_artist',
       'track_id'],
      dtype='object')

In [14]:
unpopular.dtypes

danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms           int64
explicit               bool
popularity            int64
track_name           object
track_artist         object
track_id             object
dtype: object